In [1]:
from model import Model, enc_dec_tensors
import numpy as np
import os
import tensorflow as tf

In [2]:
config_path = './config.npy'
features_dir = './features/'
model_path = './models/model99.h5'
x_path = '../data/specs-x-match.npy'

In [3]:
config = np.load(config_path).item()
spec = np.load(x_path).item()

In [4]:
def get_enc_dec(x, config, model_path):
    batch_size = x.shape[0]
    original_dim = config['original_dim']
    latent_dim = config['latent_dim']
    rate = config['rate']

    tf.reset_default_graph()
    with tf.Session() as sess:
        inputs, eps, _, _, _, _, _, _, _ = Model(batch_size, original_dim, latent_dim, rate)
        enc_mean, dec_mean = enc_dec_tensors()

        saver = tf.train.Saver()
        saver.restore(sess, model_path)

        eps_in = np.full([x.shape[0], latent_dim], 0)
        feed_dict = {inputs: x, eps: eps_in}
        enc_x, dec_x = sess.run([enc_mean, dec_mean],feed_dict=feed_dict)
        
    return enc_x, dec_x

In [5]:
x = spec['flux']
enc_x, _ = get_enc_dec(x, config, model_path)
spec['vae'] = enc_x

[3794, 2848, 1902, 956, 11]
INFO:tensorflow:Restoring parameters from ./models/model99.h5


In [6]:
features = {}
features['names'] = spec['names']
features['vae'] = spec['vae']

### Save

In [7]:
if not os.path.exists(features_dir):
    os.makedirs(features_dir)
np.save(features_dir+'features.npy', features)